### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,42651423,144553,79085,51530,0,0,1139,5,2025-05-08 18:30,2025-05-08 18:30,2,0,6142,2,1
34,42651373,32516,79085,3,0,0,1139,6,2025-05-09 19:21,2025-05-09 19:21,0,4,7652,5,1
35,42651861,1994,79085,51530,0,0,1140,0,2025-05-10 19:35,2025-05-10 19:35,1,0,7072,7,1
36,42410320,78671,79085,1349,15,70,1140,1,2025-05-11 21:59,2025-05-11 21:59,1,0,8132,1,1
37,42655423,32404,79085,51530,0,0,1140,2,2025-05-12 19:30,2025-05-12 19:30,2,0,7707,4,1
38,42655534,76562,79085,51530,0,0,1140,3,2025-05-13 09:30,2025-05-13 09:30,3,0,4305,2,1
39,42410308,79085,10520,1349,16,70,1140,4,2025-05-14 21:59,2025-05-14 21:59,0,4,15287,6,1
40,42656197,30011,79085,51530,0,0,1140,5,2025-05-15 18:30,2025-05-15 18:30,0,4,2682,3,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

8

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,144553,28454,38900,1,16,1,0,0,fff11,12,10,12,42651423
1,79085,41619,31173,2,2,5,2,0,4-3-3a,9,9,11,42651423
0,32516,42865,35244,1,5,3,1,0,Obrana,6,7,6,42651373
1,79085,27335,34956,0,16,2,0,0,4-3-3a,10,10,10,42651373
0,1994,21396,40683,1,9,0,0,0,maja,9,11,11,42651861
1,79085,48596,29309,0,6,2,0,0,5-3-2a,9,9,10,42651861
0,78671,31763,38476,1,10,3,1,0,SEM NOME,10,9,12,42410320
1,79085,38302,31589,1,5,0,0,0,5-3-2a,9,9,9,42410320
0,32404,31199,32843,3,14,3,1,0,Rambajstika 4PW,11,8,10,42655423
1,79085,38785,37141,1,3,2,2,0,5-4-1a,9,9,10,42655423


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1744

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,10520,16,41,13,2,1,67,12,41555067013011,1349
1,23775,16,40,13,1,2,47,13,40534047013001,1349
2,124139,16,33,10,3,3,40,15,33525040010006,1349
3,23755,16,29,9,2,5,46,26,29520046009010,1349
4,122169,16,26,7,5,4,37,23,26514037007005,1349
5,78671,16,26,8,2,6,28,25,26503028008008,1349
6,79866,16,23,6,5,5,34,18,23516034006009,1349
7,79085,16,19,5,4,7,26,24,19502026005004,1349
8,89569,16,18,5,3,8,26,30,18496026005007,1349
9,80293,16,9,2,3,11,13,29,9484013002003,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()